In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn_pandas import DataFrameMapper

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout, PReLU, LeakyReLU

import keras
import tensorflow as tf

from keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option("max.columns", None)

In [5]:
df = pd.read_csv("injuries_mixed_with_ranks.csv")
del df["Unnamed: 0"], df["visitor"], df["home"], df['ot'], df["id"], df["date"], df["attendance"]

In [6]:
df.head()

,start (et),pts_visitor,pts_home,visitor_rank_assists,visitor_rank_blocks,visitor_rank_deffeciency,visitor_rank_drebounds,visitor_rank_fouls,visitor_rank_ft_pct,visitor_rankoeffeciency,visitor_rank_opp_assists,visitor_rank_opp_blocks,visitor_rank_opp_drebounds,visitor_rank_opp_fouls,visitor_rank_opp_orebounds,visitor_rank_opp_ptsfastbreak,visitor_rank_opp_ptsfrom2,visitor_rank_opp_ptsfrom3,visitor_rank_opp_ptsinpaint,visitor_rank_opp_ptspergame,visitor_rank_opp_steals,visitor_rank_opp_threepointpct,visitor_rank_opp_turnovers,visitor_rank_opp_twopointpct,visitor_rank_orebounds,visitor_rank_ptsfrom2,visitor_rank_ptsfrom3,visitor_rank_ptsfromfastbreak,visitor_rank_ptsinpaint,visitor_rank_scoring,visitor_rank_steals,visitor_rank_threeptpct,visitor_rank_turnovers,visitor_rank_twoptpct,visitor_back_to_back,visitor_added_player,visitor_lost_player,home_rank_assists,home_rank_blocks,home_rank_deffeciency,home_rank_drebounds,home_rank_fouls,home_rank_ft_pct,home_rankoeffeciency,home_rank_opp_assists,home_rank_opp_blocks,home_rank_opp_drebounds,home_rank_opp_fouls,home_rank_opp_orebounds,home_rank_opp_ptsfastbreak,home_rank_opp_ptsfrom2,home_rank_opp_ptsfrom3,home_rank_opp_ptsinpaint,home_rank_opp_ptspergame,home_rank_opp_steals,home_rank_opp_threepointpct,home_rank_opp_turnovers,home_rank_opp_twopointpct,home_rank_orebounds,home_rank_ptsfrom2,home_rank_ptsfrom3,home_rank_ptsfromfastbreak,home_rank_ptsinpaint,home_rank_scoring,home_rank_steals,home_rank_threeptpct,home_rank_turnovers,home_rank_twoptpct,home_back_to_back,home_added_player,home_lost_player
0,7:00p,105,130,27,5,4,7,11,10,22,15,22,14,27,27,10,26,4,24,6,18,1,21,8,27,20,11,19,27,19,17,6,24,17,False,0.0,0.0,10,8,20,12,2,7,20,21,18,26,24,1,23,9,24,6,11,15,22,27,17,12,30,2,21,29,25,24,11,7,28,False,0.0,0.0
1,7:30p,94,82,25,21,22,29,20,13,27,10,3,13,7,24,8,25,7,26,20,28,26,5,18,25,28,15,7,30,26,4,8,27,25,False,1.0,2.0,24,29,5,10,3,30,17,5,19,19,5,2,9,17,6,20,4,12,8,15,22,1,24,14,25,17,22,28,18,9,30,False,0.0,0.0
2,7:30p,98,92,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,False,1.0,2.0,8,10,17,14,7,1,4,13,2,10,26,10,3,6,15,18,7,5,4,28,24,29,19,7,13,6,13,20,1,8,10,False,2.0,2.0
3,8:00p,103,94,13,1,13,17,22,24,25,9,29,20,25,30,12,24,13,16,24,16,5,4,16,5,8,17,9,3,9,10,22,25,24,False,0.0,0.0,18,27,29,27,14,19,18,27,13,25,6,15,25,22,30,23,30,22,27,24,28,19,11,13,16,11,10,23,14,16,19,False,0.0,1.0
4,8:30p,102,91,15,30,6,9,28,18,6,19,9,3,4,4,28,2,10,7,8,29,14,12,4,8,18,4,28,12,4,3,2,26,18,False,0.0,0.0,4,28,12,1,24,20,16,4,24,29,14,11,22,4,20,3,9,4,6,29,12,21,14,8,14,20,15,29,17,1,16,False,1.0,1.0


In [7]:
z = (df["pts_home"] > df["pts_visitor"])

In [8]:
df["home_team_won"] = 0

In [9]:
df["home_team_won"][z] = 1

In [10]:
df["home_team_won"].value_counts()

1    3528
0    2583
Name: home_team_won, dtype: int64

In [11]:
3528 / (3528+2583)

0.5773195876288659

In [12]:
df["start (et)"] = df["start (et)"].str.replace("p", "")

In [13]:
df["start (et)"] = df["start (et)"].astype(str)

In [14]:
target = "home_team_won"
X = df.drop(target, axis=1)
y = df[target]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [16]:
mapper = DataFrameMapper([
    ('start (et)', LabelBinarizer()), 
    (['visitor_rank_assists'], MinMaxScaler()), 
    (['visitor_rank_blocks'], MinMaxScaler()),
    (['visitor_rank_deffeciency'], MinMaxScaler()), 
    (['visitor_rank_drebounds'], MinMaxScaler()),
    (['visitor_rank_fouls'], MinMaxScaler()), 
    (['visitor_rank_ft_pct'], MinMaxScaler()), 
    (['visitor_rankoeffeciency'], MinMaxScaler()),
    (['visitor_rank_opp_assists'], MinMaxScaler()), 
    (['visitor_rank_opp_blocks'], MinMaxScaler()),
    (['visitor_rank_opp_drebounds'], MinMaxScaler()), 
    (['visitor_rank_opp_fouls'], MinMaxScaler()),
    (['visitor_rank_opp_orebounds'], MinMaxScaler()), 
    (['visitor_rank_opp_ptsfastbreak'], MinMaxScaler()),
    (['visitor_rank_opp_ptsfrom2'], MinMaxScaler()), 
    (['visitor_rank_opp_ptsfrom3'], MinMaxScaler()),
    (['visitor_rank_opp_ptsinpaint'], MinMaxScaler()), 
    (['visitor_rank_opp_ptspergame'], MinMaxScaler()),
    (['visitor_rank_opp_steals'], MinMaxScaler()), 
    (['visitor_rank_opp_threepointpct'], MinMaxScaler()),
    (['visitor_rank_opp_turnovers'], MinMaxScaler()), 
    (['visitor_rank_opp_twopointpct'], MinMaxScaler()),
    (['visitor_rank_orebounds'], MinMaxScaler()), 
    (['visitor_rank_ptsfrom2'], MinMaxScaler()),
    (['visitor_rank_ptsfrom3'], MinMaxScaler()), 
    (['visitor_rank_ptsfromfastbreak'], MinMaxScaler()),
    (['visitor_rank_ptsinpaint'], MinMaxScaler()), 
    (['visitor_rank_scoring'], MinMaxScaler()),
    (['visitor_rank_steals'], MinMaxScaler()), 
    (['visitor_rank_threeptpct'], MinMaxScaler()),
    (['visitor_rank_turnovers'], MinMaxScaler()), 
    (['visitor_rank_twoptpct'], MinMaxScaler()),
    ('visitor_back_to_back', LabelBinarizer()),
    (['visitor_added_player'], MinMaxScaler()),
    (['visitor_lost_player'], MinMaxScaler()),
    (['home_rank_assists'], MinMaxScaler()),
    (['home_rank_blocks'], MinMaxScaler()), 
    (['home_rank_deffeciency'], MinMaxScaler()), 
    (['home_rank_drebounds'], MinMaxScaler()),
    (['home_rank_fouls'], MinMaxScaler()), 
    (['home_rank_ft_pct'], MinMaxScaler()), 
    (['home_rankoeffeciency'], MinMaxScaler()),
    (['home_rank_opp_assists'], MinMaxScaler()), 
    (['home_rank_opp_blocks'], MinMaxScaler()),
    (['home_rank_opp_drebounds'], MinMaxScaler()), 
    (['home_rank_opp_fouls'], MinMaxScaler()),
    (['home_rank_opp_orebounds'], MinMaxScaler()), 
    (['home_rank_opp_ptsfastbreak'], MinMaxScaler()),
    (['home_rank_opp_ptsfrom2'], MinMaxScaler()), 
    (['home_rank_opp_ptsfrom3'], MinMaxScaler()),
    (['home_rank_opp_ptsinpaint'], MinMaxScaler()), 
    (['home_rank_opp_ptspergame'], MinMaxScaler()),
    (['home_rank_opp_steals'], MinMaxScaler()), 
    (['home_rank_opp_threepointpct'], MinMaxScaler()),
    (['home_rank_opp_turnovers'], MinMaxScaler()), 
    (['home_rank_opp_twopointpct'], MinMaxScaler()),
    (['home_rank_orebounds'], MinMaxScaler()), 
    (['home_rank_ptsfrom2'], MinMaxScaler()), 
    (['home_rank_ptsfrom3'], MinMaxScaler()),
    (['home_rank_ptsfromfastbreak'], MinMaxScaler()), 
    (['home_rank_ptsinpaint'], MinMaxScaler()),
    (['home_rank_scoring'], MinMaxScaler()), 
    (['home_rank_steals'], MinMaxScaler()), 
    (['home_rank_threeptpct'], MinMaxScaler()),
    (['home_rank_turnovers'], MinMaxScaler()), 
    (['home_rank_twoptpct'], MinMaxScaler()),
    ('home_back_to_back', LabelBinarizer()),
    (['home_added_player'], MinMaxScaler()),
    (['home_lost_player'], MinMaxScaler()),
],df_out=True)

In [17]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [18]:
model = LogisticRegression(max_iter=300).fit(Z_train, y_train)
print("Log regress train score is " + str(model.score(Z_train, y_train)))
print("Log regress test score is " + str(model.score(Z_test, y_test)))

Log regress train score is 0.666812131791403
Log regress test score is 0.6518324607329843


In [19]:
model = RandomForestClassifier().fit(Z_train, y_train)
print("Random Forest train score is " + str(model.score(Z_train, y_train)))
print("Random Forest test score is " + str(model.score(Z_test, y_test)))

Random Forest train score is 1.0
Random Forest test score is 0.6518324607329843


In [20]:
model = CatBoostClassifier().fit(Z_train, y_train)
print("Cat Boost train score is " + str(model.score(Z_train, y_train)))
print("Cat Boost test score is " + str(model.score(Z_test, y_test)))

Learning rate set to 0.019735
0:	learn: 0.6907794	total: 60.7ms	remaining: 1m
1:	learn: 0.6886599	total: 63.8ms	remaining: 31.8s
2:	learn: 0.6867414	total: 66.9ms	remaining: 22.2s
3:	learn: 0.6845499	total: 70ms	remaining: 17.4s
4:	learn: 0.6822073	total: 73ms	remaining: 14.5s
5:	learn: 0.6804296	total: 76.2ms	remaining: 12.6s
6:	learn: 0.6785178	total: 79.3ms	remaining: 11.2s
7:	learn: 0.6767918	total: 82.5ms	remaining: 10.2s
8:	learn: 0.6750105	total: 85.7ms	remaining: 9.43s
9:	learn: 0.6734689	total: 88.7ms	remaining: 8.78s
10:	learn: 0.6716700	total: 91.7ms	remaining: 8.24s
11:	learn: 0.6698380	total: 94.7ms	remaining: 7.8s
12:	learn: 0.6680159	total: 99.1ms	remaining: 7.52s
13:	learn: 0.6664323	total: 102ms	remaining: 7.2s
14:	learn: 0.6648137	total: 105ms	remaining: 6.91s
15:	learn: 0.6632450	total: 108ms	remaining: 6.66s
16:	learn: 0.6618100	total: 111ms	remaining: 6.44s
17:	learn: 0.6603141	total: 114ms	remaining: 6.25s
18:	learn: 0.6587644	total: 118ms	remaining: 6.07s
19:	lea

200:	learn: 0.5587762	total: 770ms	remaining: 3.06s
201:	learn: 0.5584222	total: 774ms	remaining: 3.06s
202:	learn: 0.5580436	total: 777ms	remaining: 3.05s
203:	learn: 0.5576961	total: 783ms	remaining: 3.06s
204:	learn: 0.5574306	total: 789ms	remaining: 3.06s
205:	learn: 0.5571780	total: 796ms	remaining: 3.07s
206:	learn: 0.5569100	total: 800ms	remaining: 3.06s
207:	learn: 0.5565886	total: 804ms	remaining: 3.06s
208:	learn: 0.5562412	total: 807ms	remaining: 3.05s
209:	learn: 0.5558417	total: 812ms	remaining: 3.05s
210:	learn: 0.5554402	total: 815ms	remaining: 3.05s
211:	learn: 0.5550936	total: 819ms	remaining: 3.04s
212:	learn: 0.5547031	total: 822ms	remaining: 3.04s
213:	learn: 0.5544515	total: 826ms	remaining: 3.04s
214:	learn: 0.5542221	total: 829ms	remaining: 3.03s
215:	learn: 0.5540122	total: 833ms	remaining: 3.02s
216:	learn: 0.5538364	total: 836ms	remaining: 3.02s
217:	learn: 0.5534889	total: 839ms	remaining: 3.01s
218:	learn: 0.5531461	total: 843ms	remaining: 3s
219:	learn: 0.5

360:	learn: 0.5128512	total: 1.39s	remaining: 2.46s
361:	learn: 0.5126354	total: 1.4s	remaining: 2.47s
362:	learn: 0.5123625	total: 1.41s	remaining: 2.47s
363:	learn: 0.5121213	total: 1.42s	remaining: 2.47s
364:	learn: 0.5118720	total: 1.42s	remaining: 2.47s
365:	learn: 0.5116283	total: 1.42s	remaining: 2.46s
366:	learn: 0.5113024	total: 1.43s	remaining: 2.46s
367:	learn: 0.5110161	total: 1.43s	remaining: 2.45s
368:	learn: 0.5108257	total: 1.43s	remaining: 2.45s
369:	learn: 0.5105374	total: 1.44s	remaining: 2.44s
370:	learn: 0.5103100	total: 1.44s	remaining: 2.44s
371:	learn: 0.5099920	total: 1.44s	remaining: 2.43s
372:	learn: 0.5097150	total: 1.44s	remaining: 2.43s
373:	learn: 0.5095444	total: 1.45s	remaining: 2.42s
374:	learn: 0.5091549	total: 1.45s	remaining: 2.42s
375:	learn: 0.5088931	total: 1.45s	remaining: 2.41s
376:	learn: 0.5086516	total: 1.46s	remaining: 2.41s
377:	learn: 0.5083955	total: 1.46s	remaining: 2.4s
378:	learn: 0.5081316	total: 1.46s	remaining: 2.4s
379:	learn: 0.5

524:	learn: 0.4649916	total: 1.96s	remaining: 1.78s
525:	learn: 0.4646632	total: 1.97s	remaining: 1.77s
526:	learn: 0.4643204	total: 1.97s	remaining: 1.77s
527:	learn: 0.4638607	total: 1.98s	remaining: 1.77s
528:	learn: 0.4635914	total: 1.99s	remaining: 1.77s
529:	learn: 0.4633163	total: 1.99s	remaining: 1.76s
530:	learn: 0.4628418	total: 1.99s	remaining: 1.76s
531:	learn: 0.4624796	total: 2s	remaining: 1.75s
532:	learn: 0.4622106	total: 2s	remaining: 1.75s
533:	learn: 0.4620106	total: 2s	remaining: 1.75s
534:	learn: 0.4616605	total: 2.01s	remaining: 1.74s
535:	learn: 0.4613066	total: 2.01s	remaining: 1.74s
536:	learn: 0.4610425	total: 2.02s	remaining: 1.74s
537:	learn: 0.4608432	total: 2.02s	remaining: 1.73s
538:	learn: 0.4605973	total: 2.02s	remaining: 1.73s
539:	learn: 0.4603869	total: 2.03s	remaining: 1.73s
540:	learn: 0.4601541	total: 2.03s	remaining: 1.72s
541:	learn: 0.4598747	total: 2.04s	remaining: 1.72s
542:	learn: 0.4594934	total: 2.04s	remaining: 1.72s
543:	learn: 0.4591756

740:	learn: 0.4029322	total: 2.73s	remaining: 953ms
741:	learn: 0.4026049	total: 2.73s	remaining: 950ms
742:	learn: 0.4022936	total: 2.74s	remaining: 947ms
743:	learn: 0.4019362	total: 2.74s	remaining: 944ms
744:	learn: 0.4016264	total: 2.75s	remaining: 941ms
745:	learn: 0.4013142	total: 2.75s	remaining: 938ms
746:	learn: 0.4011321	total: 2.76s	remaining: 934ms
747:	learn: 0.4009170	total: 2.76s	remaining: 930ms
748:	learn: 0.4006988	total: 2.77s	remaining: 927ms
749:	learn: 0.4004537	total: 2.77s	remaining: 924ms
750:	learn: 0.4001298	total: 2.77s	remaining: 920ms
751:	learn: 0.3998599	total: 2.78s	remaining: 916ms
752:	learn: 0.3996336	total: 2.79s	remaining: 914ms
753:	learn: 0.3994554	total: 2.79s	remaining: 910ms
754:	learn: 0.3992250	total: 2.79s	remaining: 906ms
755:	learn: 0.3990087	total: 2.8s	remaining: 904ms
756:	learn: 0.3987261	total: 2.8s	remaining: 900ms
757:	learn: 0.3985712	total: 2.81s	remaining: 896ms
758:	learn: 0.3982779	total: 2.81s	remaining: 892ms
759:	learn: 0.

902:	learn: 0.3633417	total: 3.3s	remaining: 354ms
903:	learn: 0.3631158	total: 3.3s	remaining: 351ms
904:	learn: 0.3628440	total: 3.31s	remaining: 347ms
905:	learn: 0.3626199	total: 3.31s	remaining: 344ms
906:	learn: 0.3623860	total: 3.32s	remaining: 340ms
907:	learn: 0.3621969	total: 3.32s	remaining: 337ms
908:	learn: 0.3619885	total: 3.33s	remaining: 333ms
909:	learn: 0.3617140	total: 3.33s	remaining: 329ms
910:	learn: 0.3614680	total: 3.33s	remaining: 326ms
911:	learn: 0.3612889	total: 3.34s	remaining: 322ms
912:	learn: 0.3610996	total: 3.34s	remaining: 318ms
913:	learn: 0.3608991	total: 3.34s	remaining: 315ms
914:	learn: 0.3607529	total: 3.35s	remaining: 311ms
915:	learn: 0.3605959	total: 3.35s	remaining: 307ms
916:	learn: 0.3603397	total: 3.35s	remaining: 303ms
917:	learn: 0.3601183	total: 3.35s	remaining: 300ms
918:	learn: 0.3598367	total: 3.36s	remaining: 296ms
919:	learn: 0.3596465	total: 3.36s	remaining: 292ms
920:	learn: 0.3595830	total: 3.36s	remaining: 289ms
921:	learn: 0.

In [21]:
model = Sequential([
    Input(shape=(Z_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(1, activation="relu")
])

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

early_stop = EarlyStopping(monitor="val_accuracy", patience=30, verbose=0, mode="max")
mcp_save = ModelCheckpoint('callback_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')

model.fit(Z_train, y_train,
            validation_data=(Z_test, y_test),
            epochs=200, batch_size=8,
            verbose=2, callbacks=[early_stop, mcp_save])

Epoch 1/200
573/573 - 6s - loss: 0.6949 - accuracy: 0.5808 - val_loss: 0.6494 - val_accuracy: 0.6126
Epoch 2/200
573/573 - 0s - loss: 0.6616 - accuracy: 0.6393 - val_loss: 0.6352 - val_accuracy: 0.6361
Epoch 3/200
573/573 - 0s - loss: 0.6479 - accuracy: 0.6476 - val_loss: 0.6546 - val_accuracy: 0.6276
Epoch 4/200
573/573 - 0s - loss: 0.6501 - accuracy: 0.6496 - val_loss: 0.6371 - val_accuracy: 0.6250
Epoch 5/200
573/573 - 0s - loss: 0.6489 - accuracy: 0.6404 - val_loss: 0.6398 - val_accuracy: 0.6302
Epoch 6/200
573/573 - 0s - loss: 0.6334 - accuracy: 0.6411 - val_loss: 0.6362 - val_accuracy: 0.6329
Epoch 7/200
573/573 - 0s - loss: 0.6307 - accuracy: 0.6539 - val_loss: 0.6301 - val_accuracy: 0.6401
Epoch 8/200
573/573 - 0s - loss: 0.6227 - accuracy: 0.6585 - val_loss: 0.6609 - val_accuracy: 0.6335
Epoch 9/200
573/573 - 0s - loss: 0.6266 - accuracy: 0.6389 - val_loss: 0.6403 - val_accuracy: 0.6309
Epoch 10/200
573/573 - 0s - loss: 0.6217 - accuracy: 0.6561 - val_loss: 0.6440 - val_accura

In [28]:
# Best Model
model = CatBoostClassifier().fit(Z_train, y_train)
print("Cat Boost train score is " + str(model.score(Z_train, y_train)))
print("Cat Boost test score is " + str(model.score(Z_test, y_test)))

Learning rate set to 0.019735
0:	learn: 0.6907794	total: 3.58ms	remaining: 3.58s
1:	learn: 0.6886599	total: 6.83ms	remaining: 3.41s
2:	learn: 0.6867414	total: 10ms	remaining: 3.34s
3:	learn: 0.6845499	total: 13.1ms	remaining: 3.27s
4:	learn: 0.6822073	total: 17.2ms	remaining: 3.42s
5:	learn: 0.6804296	total: 20.4ms	remaining: 3.37s
6:	learn: 0.6785178	total: 23.5ms	remaining: 3.33s
7:	learn: 0.6767918	total: 26.6ms	remaining: 3.3s
8:	learn: 0.6750105	total: 29.9ms	remaining: 3.29s
9:	learn: 0.6734689	total: 32.9ms	remaining: 3.26s
10:	learn: 0.6716700	total: 36ms	remaining: 3.23s
11:	learn: 0.6698380	total: 39ms	remaining: 3.21s
12:	learn: 0.6680159	total: 42ms	remaining: 3.19s
13:	learn: 0.6664323	total: 45.1ms	remaining: 3.18s
14:	learn: 0.6648137	total: 48.1ms	remaining: 3.16s
15:	learn: 0.6632450	total: 51.2ms	remaining: 3.15s
16:	learn: 0.6618100	total: 54.2ms	remaining: 3.13s
17:	learn: 0.6603141	total: 57.3ms	remaining: 3.12s
18:	learn: 0.6587644	total: 60.4ms	remaining: 3.12s
1

167:	learn: 0.5700820	total: 574ms	remaining: 2.84s
168:	learn: 0.5696963	total: 580ms	remaining: 2.85s
169:	learn: 0.5693676	total: 585ms	remaining: 2.85s
170:	learn: 0.5689890	total: 590ms	remaining: 2.86s
171:	learn: 0.5686270	total: 595ms	remaining: 2.87s
172:	learn: 0.5681904	total: 599ms	remaining: 2.87s
173:	learn: 0.5679128	total: 602ms	remaining: 2.86s
174:	learn: 0.5674924	total: 606ms	remaining: 2.85s
175:	learn: 0.5671998	total: 609ms	remaining: 2.85s
176:	learn: 0.5669088	total: 612ms	remaining: 2.84s
177:	learn: 0.5665905	total: 616ms	remaining: 2.84s
178:	learn: 0.5663298	total: 619ms	remaining: 2.84s
179:	learn: 0.5659280	total: 622ms	remaining: 2.83s
180:	learn: 0.5655355	total: 625ms	remaining: 2.83s
181:	learn: 0.5651835	total: 628ms	remaining: 2.82s
182:	learn: 0.5647976	total: 632ms	remaining: 2.82s
183:	learn: 0.5644549	total: 635ms	remaining: 2.81s
184:	learn: 0.5641630	total: 638ms	remaining: 2.81s
185:	learn: 0.5638639	total: 641ms	remaining: 2.8s
186:	learn: 0

344:	learn: 0.5173278	total: 1.16s	remaining: 2.2s
345:	learn: 0.5170559	total: 1.16s	remaining: 2.2s
346:	learn: 0.5167360	total: 1.17s	remaining: 2.2s
347:	learn: 0.5164831	total: 1.17s	remaining: 2.2s
348:	learn: 0.5162320	total: 1.18s	remaining: 2.2s
349:	learn: 0.5158985	total: 1.18s	remaining: 2.19s
350:	learn: 0.5156383	total: 1.19s	remaining: 2.19s
351:	learn: 0.5153245	total: 1.19s	remaining: 2.19s
352:	learn: 0.5151868	total: 1.19s	remaining: 2.19s
353:	learn: 0.5148289	total: 1.2s	remaining: 2.18s
354:	learn: 0.5145052	total: 1.2s	remaining: 2.18s
355:	learn: 0.5142959	total: 1.2s	remaining: 2.17s
356:	learn: 0.5139855	total: 1.21s	remaining: 2.17s
357:	learn: 0.5137029	total: 1.21s	remaining: 2.17s
358:	learn: 0.5135033	total: 1.21s	remaining: 2.16s
359:	learn: 0.5131419	total: 1.22s	remaining: 2.16s
360:	learn: 0.5128512	total: 1.22s	remaining: 2.16s
361:	learn: 0.5126354	total: 1.22s	remaining: 2.15s
362:	learn: 0.5123625	total: 1.22s	remaining: 2.15s
363:	learn: 0.512121

512:	learn: 0.4688289	total: 1.74s	remaining: 1.65s
513:	learn: 0.4684754	total: 1.74s	remaining: 1.65s
514:	learn: 0.4681013	total: 1.75s	remaining: 1.65s
515:	learn: 0.4678093	total: 1.75s	remaining: 1.65s
516:	learn: 0.4673717	total: 1.76s	remaining: 1.64s
517:	learn: 0.4670582	total: 1.76s	remaining: 1.64s
518:	learn: 0.4669021	total: 1.76s	remaining: 1.64s
519:	learn: 0.4665941	total: 1.77s	remaining: 1.63s
520:	learn: 0.4662489	total: 1.77s	remaining: 1.63s
521:	learn: 0.4659636	total: 1.77s	remaining: 1.63s
522:	learn: 0.4656512	total: 1.78s	remaining: 1.62s
523:	learn: 0.4652917	total: 1.78s	remaining: 1.62s
524:	learn: 0.4649916	total: 1.78s	remaining: 1.61s
525:	learn: 0.4646632	total: 1.79s	remaining: 1.61s
526:	learn: 0.4643204	total: 1.79s	remaining: 1.61s
527:	learn: 0.4638607	total: 1.79s	remaining: 1.6s
528:	learn: 0.4635914	total: 1.8s	remaining: 1.6s
529:	learn: 0.4633163	total: 1.8s	remaining: 1.59s
530:	learn: 0.4628418	total: 1.8s	remaining: 1.59s
531:	learn: 0.462

692:	learn: 0.4156576	total: 2.32s	remaining: 1.03s
693:	learn: 0.4154528	total: 2.32s	remaining: 1.02s
694:	learn: 0.4153500	total: 2.33s	remaining: 1.02s
695:	learn: 0.4151324	total: 2.33s	remaining: 1.02s
696:	learn: 0.4148782	total: 2.33s	remaining: 1.01s
697:	learn: 0.4145831	total: 2.34s	remaining: 1.01s
698:	learn: 0.4143442	total: 2.34s	remaining: 1.01s
699:	learn: 0.4142961	total: 2.34s	remaining: 1s
700:	learn: 0.4140222	total: 2.35s	remaining: 1s
701:	learn: 0.4138413	total: 2.35s	remaining: 999ms
702:	learn: 0.4135604	total: 2.35s	remaining: 995ms
703:	learn: 0.4133685	total: 2.36s	remaining: 992ms
704:	learn: 0.4131735	total: 2.36s	remaining: 988ms
705:	learn: 0.4129142	total: 2.36s	remaining: 985ms
706:	learn: 0.4125816	total: 2.37s	remaining: 981ms
707:	learn: 0.4123043	total: 2.37s	remaining: 978ms
708:	learn: 0.4120993	total: 2.37s	remaining: 974ms
709:	learn: 0.4118282	total: 2.38s	remaining: 971ms
710:	learn: 0.4115791	total: 2.38s	remaining: 967ms
711:	learn: 0.4112

870:	learn: 0.3708924	total: 2.89s	remaining: 428ms
871:	learn: 0.3706290	total: 2.9s	remaining: 425ms
872:	learn: 0.3703967	total: 2.9s	remaining: 422ms
873:	learn: 0.3701706	total: 2.91s	remaining: 419ms
874:	learn: 0.3698756	total: 2.91s	remaining: 416ms
875:	learn: 0.3698197	total: 2.92s	remaining: 413ms
876:	learn: 0.3695493	total: 2.92s	remaining: 410ms
877:	learn: 0.3693167	total: 2.93s	remaining: 407ms
878:	learn: 0.3691295	total: 2.94s	remaining: 404ms
879:	learn: 0.3688785	total: 2.94s	remaining: 402ms
880:	learn: 0.3686297	total: 2.95s	remaining: 398ms
881:	learn: 0.3683584	total: 2.95s	remaining: 395ms
882:	learn: 0.3680715	total: 2.96s	remaining: 392ms
883:	learn: 0.3679030	total: 2.96s	remaining: 388ms
884:	learn: 0.3676762	total: 2.96s	remaining: 385ms
885:	learn: 0.3674765	total: 2.97s	remaining: 382ms
886:	learn: 0.3671882	total: 2.97s	remaining: 378ms
887:	learn: 0.3669399	total: 2.97s	remaining: 375ms
888:	learn: 0.3667193	total: 2.98s	remaining: 372ms
889:	learn: 0.

In [29]:
df2 = pd.read_csv("daily_data/full_daily_ranks.csv")

In [41]:
df_away = df2[(df2["team"] == "Denver")]
del df_away["Unnamed: 0"], df_away["team"]

i = "visitor"
df_away = df_away.rename(columns={'rank_assists': str(i)+"_rank_assists",
       'rank_blocks':str(i)+"_rank_blocks", 'rank_deffeciency': str(i)+ "_rank_deffeciency", 'rank_drebounds': str(i)+ "_rank_drebounds",
       'rank_fouls': str(i)+"_rank_fouls",'rank_ft_pct': str(i)+"_rank_ft_pct", 'rank_oeffeciency': str(i)+"_rankoeffeciency", 'rank_opp_assists': str(i)+"_rank_opp_assists",
       'rank_opp_blocks': str(i)+"_rank_opp_blocks", 'rank_opp_drebounds': str(i)+"_rank_opp_drebounds", 'rank_opp_fouls': str(i)+"_rank_opp_fouls",
       'rank_opp_orebounds':str(i)+"_rank_opp_orebounds", 'rank_opp_ptsfastbreak': str(i)+"_rank_opp_ptsfastbreak", 'rank_opp_ptsfrom2': str(i)+"_rank_opp_ptsfrom2",
       'rank_opp_ptsfrom3': str(i)+"_rank_opp_ptsfrom3", 'rank_opp_ptsinpaint': str(i)+"_rank_opp_ptsinpaint", 'rank_opp_ptspergame': str(i)+"_rank_opp_ptspergame",
       'rank_opp_steals':str(i)+"_rank_opp_steals", 'rank_opp_threepointpct': str(i)+"_rank_opp_threepointpct", 'rank_opp_turnovers': str(i)+"_rank_opp_turnovers",
       'rank_opp_twopointpct': str(i)+"_rank_opp_twopointpct", 'rank_orebounds': str(i)+"_rank_orebounds", 'rank_ptsfrom2': str(i)+"_rank_ptsfrom2",
       'rank_ptsfrom3': str(i)+"_rank_ptsfrom3", 'rank_ptsfromfastbreak':str(i)+"_rank_ptsfromfastbreak", 'rank_ptsinpaint': str(i)+"_rank_ptsinpaint",
       'rank_scoring': str(i)+"_rank_scoring", 'rank_steals':str(i)+"_rank_steals", 'rank_threeptpct':str(i)+"_rank_threeptpct", 'rank_turnovers': str(i)+"_rank_turnovers",
       'rank_twoptpct':str(i)+"_rank_twoptpct"})

df_away = df_away.reset_index()

df_home = df2[(df2["team"] == "LA Clippers")]
del df_home["Unnamed: 0"], df_home["team"]

i = "home"
df_home = df_home.rename(columns={'rank_assists': str(i)+"_rank_assists",
       'rank_blocks':str(i)+"_rank_blocks", 'rank_deffeciency': str(i)+ "_rank_deffeciency", 'rank_drebounds': str(i)+ "_rank_drebounds",
       'rank_fouls': str(i)+"_rank_fouls",'rank_ft_pct': str(i)+"_rank_ft_pct", 'rank_oeffeciency': str(i)+"_rankoeffeciency", 'rank_opp_assists': str(i)+"_rank_opp_assists",
       'rank_opp_blocks': str(i)+"_rank_opp_blocks", 'rank_opp_drebounds': str(i)+"_rank_opp_drebounds", 'rank_opp_fouls': str(i)+"_rank_opp_fouls",
       'rank_opp_orebounds':str(i)+"_rank_opp_orebounds", 'rank_opp_ptsfastbreak': str(i)+"_rank_opp_ptsfastbreak", 'rank_opp_ptsfrom2': str(i)+"_rank_opp_ptsfrom2",
       'rank_opp_ptsfrom3': str(i)+"_rank_opp_ptsfrom3", 'rank_opp_ptsinpaint': str(i)+"_rank_opp_ptsinpaint", 'rank_opp_ptspergame': str(i)+"_rank_opp_ptspergame",
       'rank_opp_steals':str(i)+"_rank_opp_steals", 'rank_opp_threepointpct': str(i)+"_rank_opp_threepointpct", 'rank_opp_turnovers': str(i)+"_rank_opp_turnovers",
       'rank_opp_twopointpct': str(i)+"_rank_opp_twopointpct", 'rank_orebounds': str(i)+"_rank_orebounds", 'rank_ptsfrom2': str(i)+"_rank_ptsfrom2",
       'rank_ptsfrom3': str(i)+"_rank_ptsfrom3", 'rank_ptsfromfastbreak':str(i)+"_rank_ptsfromfastbreak", 'rank_ptsinpaint': str(i)+"_rank_ptsinpaint",
       'rank_scoring': str(i)+"_rank_scoring", 'rank_steals':str(i)+"_rank_steals", 'rank_threeptpct':str(i)+"_rank_threeptpct", 'rank_turnovers': str(i)+"_rank_turnovers",
       'rank_twoptpct':str(i)+"_rank_twoptpct"})

df_home = df_home.reset_index()

df_predict = pd.concat([df_away, df_home], axis=1)
del df_predict["index"]

df_predict["start (et)"] = "10:00"
df_predict["visitor_added_player"] = 1
df_predict["visitor_lost_player"] = 0
df_predict["visitor_back_to_back"] = False
df_predict["home_added_player"] = 0
df_predict["home_lost_player"] = 0
df_predict["home_back_to_back"] = False

Z_new = mapper.transform(df_predict)
model.predict(Z_new)

array([1])